# Details of the BioDendro pipeline

## Running the pipeline manually

BioDendro essentially does the following steps:

1. Parse the MGF and components files
   - Optionally scaling and filtering Ions from MGF with a minimum intensity threshold.
2. Find Ions matching components
3. Bins mz values by proximity
4. Uses binned mz values to cluster components into a tree

In [1]:
# Load required modules

import os
import plotly
from BioDendro import preprocess
from BioDendro import cluster

Loading the MGF file is relatively straight forward using python.

The pipeline changes the MGF record titles in a way that won't always be portable.
If you encounter issues with record titles, you may wish to exclude the `split_msms_title` step.

In [2]:
# Load the MSMS records

with open("MSMS.mgf") as handle:
    mgf = preprocess.MGF.parse(handle)

# Splits the title up.
# You may need to skip this step.
for record in mgf.records:
    record.title = preprocess.split_msms_title(record.title)

mgf

MGF(records=[
MGFRecord(title='QE_2017_001814', retention=141.0, pepmass=Ion(mz=81.52061, intensity=1678597.25), charge='1+', ions=[Ion(mz=53.00264, intensity=1252.15), Ion(mz=60.7764, intensity=1324.65)]),
MGFRecord(title='QE_2017_001814', retention=877.0, pepmass=Ion(mz=81.52061, intensity=19463956.0), charge='2+', ions=[Ion(mz=59.32066, intensity=14996.5), Ion(mz=131.81848, intensity=18682.3), Ion(mz=176.21725, intensity=18992.1)]),
MGFRecord(title='QE_2017_001816', retention=447.0, pepmass=Ion(mz=81.52061, intensity=2814306.5), charge='1+', ions=[Ion(mz=53.0393, intensity=1853.67)]),
MGFRecord(title='QE_2017_001814', retention=436.0, pepmass=Ion(mz=81.52064, intensity=2750625.5), charge='1+', ions=[Ion(mz=63.59929, intensity=2020.82)]),
MGFRecord(title='QE_2017_001815', retention=887.0, pepmass=Ion(mz=81.52064, intensity=15775805.0), charge='2+', ions=[Ion(mz=124.5074, intensity=13727.9), Ion(mz=184.32289, intensity=14260.0)])
...
])

At this point we might decide to scale and/or filter the Ions for each MGFRecord.

In [3]:
with open("MSMS.mgf") as handle:
    mgf_scaled = preprocess.MGF.parse(handle, scaling=True, filtering=True, eps=0.2)

# Splits the title up.
# You may need to skip this step.
for record in mgf_scaled.records:
    record.title = preprocess.split_msms_title(record.title)

mgf_scaled

MGF(records=[
MGFRecord(title='QE_2017_001814', retention=141.0, pepmass=Ion(mz=81.52061, intensity=1678597.25), charge='1+', ions=[Ion(mz=53.00264, intensity=0.9452685615068132), Ion(mz=60.7764, intensity=1.0)]),
MGFRecord(title='QE_2017_001814', retention=877.0, pepmass=Ion(mz=81.52061, intensity=19463956.0), charge='2+', ions=[Ion(mz=59.32066, intensity=0.7896177884488814), Ion(mz=131.81848, intensity=0.9836879544652777), Ion(mz=176.21725, intensity=1.0)]),
MGFRecord(title='QE_2017_001816', retention=447.0, pepmass=Ion(mz=81.52061, intensity=2814306.5), charge='1+', ions=[Ion(mz=53.0393, intensity=1.0)]),
MGFRecord(title='QE_2017_001814', retention=436.0, pepmass=Ion(mz=81.52064, intensity=2750625.5), charge='1+', ions=[Ion(mz=63.59929, intensity=1.0)]),
MGFRecord(title='QE_2017_001815', retention=887.0, pepmass=Ion(mz=81.52064, intensity=15775805.0), charge='2+', ions=[Ion(mz=124.5074, intensity=0.9626858345021038), Ion(mz=184.32289, intensity=1.0)])
...
])

You'll notice in the ions field of each MGF record that the "intensity" values have been scaled to numbers between 0 and 1, and that any ions with scaled intensity below 0.2 will be filtered out.

Loading components files is a similar process.

In [4]:
# Load the list of components to compare to the MSMS file

with open("./component_list.txt") as handle:
    components = preprocess.SampleRecord.parse(handle)

components[:5]

[SampleRecord(mz=125.9862, retention=56.964, original='Chinese Spring 1_001829_0849_m/z125.9862_RT0.9494'),
 SampleRecord(mz=129.1273, retention=318.834, original='Chinese Spring 1_001829_4030_m/z129.1273_RT5.3139'),
 SampleRecord(mz=129.1273, retention=348.516, original='Chinese Spring 1_001829_4681_m/z129.1273_RT5.8086'),
 SampleRecord(mz=129.1274, retention=327.924, original='Chinese Spring 1_001829_4250_m/z129.1274_RT5.4654'),
 SampleRecord(mz=129.1274, retention=392.68800000000005, original='Chinese Spring 1_001829_5303_m/z129.1274_RT6.5448')]

Next we find the ions for each components.

In [5]:
# Remove redundant records with mass and retention time tolerance 

df = preprocess.remove_redundancy(components, mgf, mz_tol=0.002, retention_tol=5, neutral=False)
df.head()

,component,sample,mz
0,Chinese Spring 1_001829_6995_m/z353.2686_RT10....,QE_2017_001814_353.26889_609.0,50.06792
1,Chinese Spring 1_001829_2250_m/z177.0544_RT3.6957,QE_2017_001816_177.05449_217.0,50.44534
2,Chinese Spring 1_001829_2318_m/z235.1439_RT3.7670,QE_2017_001815_235.14398_226.0,50.48630
3,Chinese Spring 1_001829_5175_m/z219.1014_RT6.3621,QE_2017_001816_219.10153_382.0,50.49461
4,Chinese Spring 1_001829_5465_m/z211.1690_RT6.7722,QE_2017_001816_211.16916_405.0,50.49868


And finally, we can cluster the data using a `Tree` object.

The interface to `Tree` is similar to `scikit-learn` objects, where first you initialise a model with parameters, then you fit it with some data.

In [6]:
# Using the non-redundant dataframe, bin analytes on threshold=8e-4 and return a data matrix
# Cluster the data matrix using clustering_method="jaccard"
# Set threshold to color dendrogram and output clusters at cutoff=0.4

tree = cluster.Tree(threshold=8e-4, clustering_method="jaccard", cutoff=0.4)
tree.fit(df)

The threshold is the parameter that affects binning.
Increasing the threshold will reduce the number of bins, decreasing the threshold will make bins more granular.

The tree object now contains all of our intermediate data.

We can look at which samples have which bins in a [Pandas](https://pandas.pydata.org/) dataframe.
This is the matrix used to compute distances based on presence absence.

In [7]:
# One hot encoded matrix output from mz binning.
tree.onehot_df.head()

bins,100.0246_100.0246_100.0246,100.0395_100.0395_100.0395,100.0759_100.0759_100.0760,100.1123_100.1122_100.1123,100.1311_100.1311_100.1311,100.5562_100.5562_100.5562,100.6662_100.6662_100.6662,101.0235_101.0235_101.0235,101.0598_101.0596_101.0599,101.0712_101.0711_101.0712,...,98.0966_98.0964_98.0967,98.5142_98.5142_98.5142,98.5942_98.5942_98.5942,98.8081_98.8081_98.8081,98.9756_98.9755_98.9759,98.9844_98.9844_98.9845,99.0034_99.0034_99.0034,99.0443_99.0441_99.0445,99.0807_99.0804_99.0809,99.1172_99.1172_99.1172
component,,,,,,,,,,,,,,,,,,,,,
Chinese Spring 1_001829_0008_m/z199.9878_RT0.5260,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Chinese Spring 1_001829_0094_m/z265.0154_RT0.5705,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Chinese Spring 1_001829_0683_m/z229.1545_RT0.7845,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Chinese Spring 1_001829_0724_m/z241.1544_RT0.7890,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
Chinese Spring 1_001829_0764_m/z229.1545_RT0.8469,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


NB. "one-hot" encoding is a common technique in ML/statistice where 1 (or True) denotes presence and 0 (or False) denotes absence.
It's related to the concept of Dummy coding.

We can also look at the clusters derived from the computed tree.

In [8]:
# Clusters derived from the tree using cutoff=0.4
tree.clusters

array([ 75, 158, 156, 121, 157, 148, 122, 135, 149, 132, 144,  36,  37,
       156, 156,  90, 133, 154, 129, 114, 126, 128, 155, 130, 127,  91,
       141,  99,  97,  98, 101, 112, 110, 136,  73, 109, 100, 102, 152,
        96, 140,  97,  96, 153, 140, 109,  87, 108, 111,  77, 131,  75,
       143,  58,  52,  58,  74,  59,  79, 105, 150,  73,  78, 147,  52,
       104, 137,  82,  84,  52,  67,  76, 151, 104,  55, 145,  48, 146,
        93,  71,  80,  81,  93,  69, 106, 107,  83,  68,  61,  61, 142,
        92,  49,  94,  61, 139,  61,  65, 125,  89,  53,  62, 123,  54,
       123,  66, 124,  81, 138, 117, 116, 116, 118, 120,  72,  45,  45,
        57,   7,   4,   9,  16,   6,  11,   8,   6,  19,   7,   9,  16,
       119,   9,   6,   8,  12,  46,  32,  27,  47,  24,  20,  29,  28,
        17,  26,  22,  30,  18,   6,   1,  31,   6,  25,  33,   3,  60,
        34,  23,  19,  35,   2, 159,  85, 113, 128, 134,  76,  76,  70,
        64,  64,  60,  88,  74,  50, 129,   5,  51,  65,  10,  1

This is a [numpy](https://www.numpy.org/) array.
The order of these is the same as the row index in the one-hot dataframe.

So we can obtain clusters for each sample like so...

In [9]:
dict(zip(tree.onehot_df.index, tree.clusters))['Chinese Spring 1_001829_0724_m/z241.1544_RT0.7890']

121

As with the python pipeline in the quick-start notebook, we can easily choose a new cutoff for clustering.
In fact, the pipeline function returns a tree.

In [10]:
# To pick new clusters without computing a new tree...
print("BEFORE: Cutoff:", tree.cutoff, "n clusters:", len(set(tree.clusters)))

tree.cut_tree(cutoff=0.6)
print("AFTER: Cutoff:", tree.cutoff, "n clusters:", len(set(tree.clusters)))

# Note that the object's cutoff value is changed.

BEFORE: Cutoff: 0.4 n clusters: 160
AFTER: Cutoff: 0.6 n clusters: 113


To get the cluster numbers along-side the presence absence data we can simply add a new colum to the dataframe, 
since it's already in the correct order.

In [11]:
# Make sure you take a copy to avoid editing the data.
oh = tree.onehot_df.copy()
oh["cluster"] = tree.clusters

# reorder columns
oh = oh[["cluster"] + [col for col in oh.columns if col != "cluster"]]

# Sort values by cluster
oh.sort_values(by="cluster", inplace=True)

oh.head()

bins,cluster,100.0246_100.0246_100.0246,100.0395_100.0395_100.0395,100.0759_100.0759_100.0760,100.1123_100.1122_100.1123,100.1311_100.1311_100.1311,100.5562_100.5562_100.5562,100.6662_100.6662_100.6662,101.0235_101.0235_101.0235,101.0598_101.0596_101.0599,...,98.0966_98.0964_98.0967,98.5142_98.5142_98.5142,98.5942_98.5942_98.5942,98.8081_98.8081_98.8081,98.9756_98.9755_98.9759,98.9844_98.9844_98.9845,99.0034_99.0034_99.0034,99.0443_99.0441_99.0445,99.0807_99.0804_99.0809,99.1172_99.1172_99.1172
component,,,,,,,,,,,,,,,,,,,,,
Chinese Spring 1_001829_8737_m/z613.4826_RT13.9777,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Chinese Spring 1_001829_7933_m/z792.5621_RT11.9987,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Chinese Spring 1_001829_8203_m/z613.4826_RT12.2884,2,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Chinese Spring 1_001829_6867_m/z596.3096_RT9.9999,3,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Chinese Spring 5_001887_7848_m/z792.5621_RT14.4273,4,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


We can write out the results of the clustering as we saw in the quick-start notebook.
Note that the `path` will be a directory where multiple files are written, and it must already exist.

Also, any results already in the directory will be overwritten.
The default path will use a combination of the date and time to avoid overwriting data.

In [12]:
# Create the directory to store results.
os.makedirs("results", exist_ok=True)

# Generate the plots of clusters.
tree.write_summaries(path="results")

In [13]:
%%bash

ls -l results | head

total 18272
-rw-r--r--. 1 darcyabjones darcyabjones   71581 Apr 24 17:22 cluster_100_1.png
-rw-r--r--. 1 darcyabjones darcyabjones    5855 Apr 24 17:22 cluster_100_1.xlsx
-rw-r--r--. 1 darcyabjones darcyabjones   57123 Apr 24 17:22 cluster_101_1.png
-rw-r--r--. 1 darcyabjones darcyabjones    5766 Apr 24 17:22 cluster_101_1.xlsx
-rw-r--r--. 1 darcyabjones darcyabjones   68087 Apr 24 17:22 cluster_102_1.png
-rw-r--r--. 1 darcyabjones darcyabjones    5852 Apr 24 17:22 cluster_102_1.xlsx
-rw-r--r--. 1 darcyabjones darcyabjones  188951 Apr 24 17:21 cluster_10_2.png
-rw-r--r--. 1 darcyabjones darcyabjones    7393 Apr 24 17:21 cluster_10_2.xlsx
-rw-r--r--. 1 darcyabjones darcyabjones   83151 Apr 24 17:22 cluster_103_2.png


We can inspect the hierarchical clustering linkage map output from scipy.
Unfortunately, this lacks a bit of intuition.
The first two columns refer to two nodes in the tree.
The third column gives the distance between the two nodes.
And the fourth column gives the number of leaves that are in or beneath these nodes.

In [14]:
# Linkage tree from scipy.
tree.tree[:6]

array([[1.30000000e+01, 1.40000000e+01, 0.00000000e+00, 2.00000000e+00],
       [1.02000000e+02, 1.04000000e+02, 0.00000000e+00, 2.00000000e+00],
       [1.10000000e+02, 1.11000000e+02, 0.00000000e+00, 2.00000000e+00],
       [1.94000000e+02, 1.95000000e+02, 0.00000000e+00, 2.00000000e+00],
       [7.10000000e+01, 1.66000000e+02, 1.81818182e-01, 2.00000000e+00],
       [9.40000000e+01, 9.60000000e+01, 1.92307692e-01, 2.00000000e+00]])

The tree is probably more useful to you.
You can write an interactive tree to a file using the `plot` method on the `Tree` object.

In [15]:
# To write out the tree.
# NB the results folder already exists from the previous step where we wrote summaries.
tree.plot(filename="results/simple_dendrogram.html", width=1000, height=1000);

In [16]:
%%bash
ls -la results/simple_dendrogram.html

-rw-r--r--. 1 darcyabjones darcyabjones 3155698 Apr 24 17:22 results/simple_dendrogram.html


Note that the width and height values are in pixels.
If your sample names are fairly long like ours are, you might like to fiddle with the height.
You can navigate to the .html file and open it in your favourite browser.

Since you're already using a Jupyter notebook you might like to plot the tree inline with your other analysis.
We can do that using some plotly functionality.

The `plot` method returns a Plotly `Figure` object which you can use with normal plo`ly functions.
So we can use the `notebook_mode` to show it inline.

In [17]:
# Make it a bit smaller this time
iplot = tree.plot(width=800, height=700)

# for visualising plot inline
# You only need to set connected to True once per notebook,
# but doing it multiple times won't hurt.
plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot(iplot)

Rolling over the leaf nodes with your mouse you'll be able to see which cluster each component belongs to, so you can then track it down in your results folder.

The `plot` method is just a wrapper around the `dendrogram` function in `BioDendro.plot.dendrogram`.
If you want to customise your plot e.g. with custom titles, you can do that there.

The `dendrogram` function is a modified version of plotlys [`figure_factory.create_dendrogram`](https://plot.ly/python/dendrogram/) function.
It is modified to reduce complexity, to handle text-rollover and to allow us to provide precomputed trees rather than having to recompute the tree each time (possibly yielding different trees and different clusters).

In [18]:
from BioDendro.plot import dendrogram

iplot = dendrogram(tree, width=800, height=700, title="TreesRCool", xlabel="Samples", ylabel="Distance")
plotly.offline.iplot(iplot)

There are some features that we're working on.

In dendrogram, the `hovertext` parameter allows you to specify arbitrary text for when you hover over the data.
However, the current tree drawing algorithm reorders the internal nodes, and we can't find the correct order for internal node data from the output of that function.
For now know that it exists, and if you figure it out, please let us know!